In [2]:
import boto3, re, sys, math, json, os, sagemaker, urllib.request
import io
import sagemaker
from sagemaker import get_execution_role
from IPython.display import Image
from IPython.display import display
from time import gmtime, strftime
from pyathena import connect
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


In [3]:
sess = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

In [42]:
ingest_create_athena_db_passed = False

In [5]:
# set a database name
database_name = "inspection"

In [6]:
# Set S3 staging directory -- this is a temporary directory used for Athena queries
s3_staging_dir = "s3://{0}/athena/staging".format(bucket)

In [7]:
conn = connect(region_name=region, s3_staging_dir=s3_staging_dir)

In [46]:
statement = "CREATE DATABASE IF NOT EXISTS {}".format(database_name)
print(statement)
pd.read_sql(statement, conn)

CREATE DATABASE IF NOT EXISTS inspection


""


In [8]:
statement = "SHOW DATABASES"

df_show = pd.read_sql(statement, conn)
df_show.head(5)

,database_name
0,default
1,inspection


In [48]:
if database_name in df_show.values:
    ingest_create_athena_db_passed = True

In [49]:
%store ingest_create_athena_db_passed

Stored 'ingest_create_athena_db_passed' (bool)


In [50]:
%store

Stored variables and their in-db values:
ingest_create_athena_db_passed             -> True
s3_private_path_tsv                        -> 's3://sagemaker-us-east-1-373893336492/inspections
s3_public_path_tsv                         -> 's3://inspections-pds'
setup_dependencies_passed                  -> True
setup_iam_roles_passed                     -> True
setup_s3_bucket_passed                     -> True


In [51]:
ingest_create_athena_tsv_passed = False

In [52]:
try:
    ingest_create_athena_db_passed
except NameError:
    print("ERROR YOU HAVE TO CREATE THE DATABASE")

In [53]:
print(ingest_create_athena_db_passed)

True


In [17]:
s3_private_path_tsv = "s3://{}/".format(bucket)

print(s3_private_path_tsv)

s3://sagemaker-us-east-1-373893336492/


In [82]:
statement1 = """DROP TABLE IF EXISTS inspection.inspect"""
statement2 = """DROP TABLE IF EXISTS inspection.buildings"""
statement3 = """DROP TABLE IF EXISTS inspection.registered"""
statement4 = """DROP TABLE IF EXISTS inspection.district"""

In [83]:
pd.read_sql(statement1, conn)
pd.read_sql(statement2, conn)
pd.read_sql(statement3, conn)
pd.read_sql(statement4, conn)

""


In [84]:
# SQL statement to execute
inspect_path = "s3://{}/inspections-pds/inspections/".format(bucket)

table_name = 'inspect'

statement = """CREATE EXTERNAL TABLE IF NOT EXISTS {}.{}(
                business_id string,
                business_name string,
                business_address string,
                inspection_date date,
                inspection_score int,
                violation_description string,
                risk_category string,
                sup_dist string
                )
                ROW FORMAT DELIMITED
                FIELDS TERMINATED BY '\\t'
                LINES TERMINATED BY '\\n'
                LOCATION '{}'
                TBLPROPERTIES ('skip.header.line.count'='1')""".format(
                    database_name, table_name, inspect_path)

print(statement)

CREATE EXTERNAL TABLE IF NOT EXISTS inspection.inspect(
                business_id string,
                business_name string,
                business_address string,
                inspection_date date,
                inspection_score int,
                violation_description string,
                risk_category string,
                sup_dist string
                )
                ROW FORMAT DELIMITED
                FIELDS TERMINATED BY '\t'
                LINES TERMINATED BY '\n'
                LOCATION 's3://sagemaker-us-east-1-373893336492/inspections-pds/inspections/'
                TBLPROPERTIES ('skip.header.line.count'='1')


In [85]:
pd.read_sql(statement, conn)

""


In [86]:
# SQL statement to execute
dist_path = "s3://{}/inspections-pds/district/".format(bucket)
table_name = 'district'

statement = """CREATE EXTERNAL TABLE IF NOT EXISTS {}.{}(
                sup_name string,
                sup_dist string
                )
                ROW FORMAT DELIMITED
                FIELDS TERMINATED BY '\\t'
                LINES TERMINATED BY '\\n'
                LOCATION '{}'
                TBLPROPERTIES ('skip.header.line.count'='1')""".format(
                    database_name, table_name, dist_path)

print(statement)

CREATE EXTERNAL TABLE IF NOT EXISTS inspection.district(
                sup_name string,
                sup_dist string
                )
                ROW FORMAT DELIMITED
                FIELDS TERMINATED BY '\t'
                LINES TERMINATED BY '\n'
                LOCATION 's3://sagemaker-us-east-1-373893336492/inspections-pds/district/'
                TBLPROPERTIES ('skip.header.line.count'='1')


In [87]:
pd.read_sql(statement, conn)

""


In [88]:
table_name = 'buildings'
buildings_path = "s3://{}/inspections-pds/build/".format(bucket)

statement = """CREATE EXTERNAL TABLE IF NOT EXISTS {}.{}(
                building_address string,
                year_built string,
                analysis_neighborhood string,
                sup_district string
                )
                ROW FORMAT DELIMITED
                FIELDS TERMINATED BY '\\t'
                LOCATION '{}'
                TBLPROPERTIES ('skip.header.line.count'='1')""".format(
                    database_name, table_name, buildings_path)

print(statement)

CREATE EXTERNAL TABLE IF NOT EXISTS inspection.buildings(
                building_address string,
                year_built string,
                analysis_neighborhood string,
                sup_district string
                )
                ROW FORMAT DELIMITED
                FIELDS TERMINATED BY '\t'
                LOCATION 's3://sagemaker-us-east-1-373893336492/inspections-pds/build/'
                TBLPROPERTIES ('skip.header.line.count'='1')


In [89]:
pd.read_sql(statement, conn)

""


In [90]:
table_name = 'registered'
buildings_path = "s3://{}/inspections-pds/businesses/".format(bucket)

statement = """CREATE EXTERNAL TABLE IF NOT EXISTS {}.{}(
                street_address string,
                business_start_date date,
                location_start_date date,
                dba_name string
                )
                ROW FORMAT DELIMITED
                FIELDS TERMINATED BY '\\t'
                LOCATION '{}'
                TBLPROPERTIES ('skip.header.line.count'='1')""".format(
                    database_name, table_name, buildings_path)

In [91]:
pd.read_sql(statement, conn)

""


In [92]:
statement = "show tables in {}".format(database_name)
pd.read_sql(statement, conn)

,tab_name
0,buildings
1,district
2,inspect
3,registered


In [93]:
table_name = 'inspect'
database_name = 'inspection'
statement = """select *
from {}.{}
LIMIT 5
""".format(database_name, table_name)

print(statement)

select *
from inspection.inspect
LIMIT 5



In [94]:
df = pd.read_sql(statement, conn)
df

,business_id,business_name,business_address,inspection_date,inspection_score,violation_description,risk_category,sup_dist
0,33084,"Miette Cakes, LLC",449 Octavia St,2019-01-02,90,Improper food labeling or menu misrepresentation,Low Risk,11
1,34181,Coffee Bar 1 LLC,1890 Bryant St #C,2019-01-02,86,Improper or defective plumbing,Low Risk,2
2,35805,Hard Knox Cafe,2448 Clement,2019-01-02,90,Wiping cloths not clean or properly stored or ...,Low Risk,4
3,876,"PATPONG THAI CUISINE, INC.",2415 CLEMENT St,2019-01-02,74,High risk vermin infestation,High Risk,4
4,2233,The AA Bakery & Cafe,1068 Stockton St,2019-01-02,90,Improper food storage,Low Risk,3


In [95]:
table_name = 'district'
database_name = 'inspection'
statement = """select *
from {}.{}
LIMIT 5
""".format(database_name, table_name)

print(statement)

select *
from inspection.district
LIMIT 5



In [50]:
df = pd.read_sql(statement, conn)
df

,sup_name
0,Chyanne Chen
1,Shamann Walton
2,Jackie Fielder
3,Joel Engardio
4,Danny Sauter


In [62]:
statement = """select i.business_id
, i.business_name
, i.business_address
, i.inspection_date
, i.inspection_score
, i.violation_description
, i.risk_category
, b.year_built
, b.analysis_neighborhood
, s.sup_dist
, s.sup_name
, r.location_start_date
, r.business_start_date
from inspection.inspect i
left join inspection.buildings b
on i.business_address = b.building_address
left join inspection.registered r
on i.business_address = r.street_address
left join inspection.district s
on i.sup_dist = s.sup_dist
LIMIT 5"""

print(statement)

select i.business_id
, i.business_name
, i.business_address
, i.inspection_date
, i.inspection_score
, i.violation_description
, i.risk_category
, b.year_built
, b.analysis_neighborhood
, s.sup_dist
, s.sup_name
, r.location_start_date
, r.business_start_date
from inspection.inspect i
left join inspection.buildings b
on i.business_address = b.building_address
left join inspection.registered r
on i.business_address = r.street_address
left join inspection.district s
on i.sup_dist = s.sup_dist
LIMIT 5


In [63]:
data = pd.read_sql(statement, conn)
data

,business_id,business_name,business_address,inspection_date,inspection_score,violation_description,risk_category,year_built,analysis_neighborhood,sup_dist,sup_name,location_start_date,business_start_date
0,33084,"Miette Cakes, LLC",449 Octavia St,2019-01-02,90,Low Risk,11,None,None,None,None,None,None
1,34181,Coffee Bar 1 LLC,1890 Bryant St #C,2019-01-02,86,Low Risk,2,None,None,None,None,None,None
2,35805,Hard Knox Cafe,2448 Clement,2019-01-02,90,Low Risk,4,None,None,None,None,None,None
3,876,"PATPONG THAI CUISINE, INC.",2415 CLEMENT St,2019-01-02,74,High Risk,4,None,None,None,None,None,None
4,2233,The AA Bakery & Cafe,1068 Stockton St,2019-01-02,90,Low Risk,3,None,None,None,None,None,None


In [65]:
data2 = data.replace('None', 'na')
print(data2)

  business_id               business_name   business_address inspection_date  \
0       33084           Miette Cakes, LLC     449 Octavia St      2019-01-02   
1       34181            Coffee Bar 1 LLC  1890 Bryant St #C      2019-01-02   
2       35805              Hard Knox Cafe       2448 Clement      2019-01-02   
3         876  PATPONG THAI CUISINE, INC.    2415 CLEMENT St      2019-01-02   
4        2233        The AA Bakery & Cafe   1068 Stockton St      2019-01-02   

   inspection_score violation_description risk_category year_built  \
0                90              Low Risk            11       None   
1                86              Low Risk             2       None   
2                90              Low Risk             4       None   
3                74             High Risk             4       None   
4                90              Low Risk             3       None   

  analysis_neighborhood sup_dist sup_name location_start_date  \
0                  None     None 